

<font color='red'>**WARNING!**</font> <br>
 <font color='red'>**---------------------------------------------------------------------------------------------**</font> <br>
 <font color='red'>**DO NOT RUN THIS NOTEBOOK LOCALLY**</font> <br>
  <font color='lime'>**IT MUST BE RUN  USING  GOOGLE COLAB**</font> <a href="https://colab.research.google.com/github/deivid-01/Snakes-Classifier/blob/main/04_model_arquitecture_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> <br/>



This Notebook is part 4  of Deep Learning Project : [Snake Classifier](https://github.com/deivid-01/Snakes-Classifier) of the course [Fundamentos de Deep Learning](https://rramosp.github.io/2021.deeplearning/intro.html) <br>

 <font color='red'>**---------------------------------------------------------------------------------------------**</font> <br>

# Model arquitecture and training


### About:
- This notebook focus on train and validate machine learning model.
---
David Andrés Torres Betancour <br/>
Computer Engineering  Student <br/>
University of Antioquia <br/>
davida.torres@udea.edu.co </br> 
<a href="https://github.com/deivid-01" target="_parent"><img src="https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white" alt="Github"/></a>


## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf

from tensorflow.keras.layers import Flatten, Dense,Dropout
from tensorflow.keras.models  import Model,load_model
from keras.applications.vgg16 import preprocess_input

from keras.applications.vgg16 import preprocess_input
#from keras.applications import vgg16
from tensorflow.keras.applications import VGG16 # Wtf ?
from keras.optimizers import Adam

from google.colab import files
import h5py

%matplotlib inline

## Tools

### Kaggle Tools

In [2]:
class bcolors:
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

def loadCredentials():
  '''
  Description:
    Loads and configure kaggle.json file with credentials.

  Parameters:
      No parameters.

  Returns:
      No return.   
  ''' 
  print(bcolors.BOLD + "Upload kaggle.json file with credentials\n" + bcolors.ENDC)
  print(bcolors.WARNING + "IMPORTANT: Filename must be: kaggle.json\n" + bcolors.ENDC)
  uploaded_file=files.upload() #Upload file
  assert len(uploaded_file)>0,"Fetch canceled: Dataset not loaded"
  assert list(uploaded_file.keys())[0]=='kaggle.json', "Filename must be: kaggle.json"
 


  os.environ['KAGGLE_CONFIG_DIR'] = '/content' #Setup folder
  ! chmod 600 /content/kaggle.json  #Hide kaggle.json


def fetchDatasetFromKaggle(dataset_name, force_fetch=False):
  '''
  Description:
    fetchs dataset  from kaggle.

  Parameters:
      dataset_name: Dataset name
      force_fetch: Remove local data and download data
  Returns:
      No return.   
  '''
  if os.path.exists('kaggle.json') == False: 
    loadCredentials() 

  if force_fetch==True:
   process_info=! kaggle datasets download -d deividt/{dataset_name} --force #Download dataset
   if "100%" in list(process_info)[-1]:
       print(bcolors.OKGREEN + "Data from kaggle successfully fetched\n" + bcolors.ENDC)
       print(bcolors.BOLD + "Unzipping data..." + bcolors.ENDC)
       ! unzip \*.zip && rm *.zip
       print(bcolors.OKGREEN + "Data is ready in your local folder!\n" + bcolors.ENDC)
   else:
       assert False,list(process_info)[-1]


  else:
   process_info =  ! kaggle datasets download -d deividt/{dataset_name} 
   print(process_info)
   if "Skipping" in list(process_info)[0]:
       print(bcolors.WARNING + "Data already exists locally\nIf you want force fetch set force_fetch parameter to True" + bcolors.ENDC)
  
   elif "100%" in list(process_info)[-1]:
       print(bcolors.OKGREEN + "Data from kaggle successfully fetch\n" + bcolors.ENDC)
       print(bcolors.BOLD + "Unzipping data... \n" + bcolors.ENDC)
       ! unzip \*.zip && rm *.zip #Unzip downloaded file and delete zip
       print(bcolors.OKGREEN + "Data is ready in your local folder!\n" + bcolors.ENDC)
   else:
       assert False,list(process_info)[-1]



def fetchModelsFromKaggle():
  '''
  Description:
    Download pretrained models from kaggle
  Parameters:
    No parameters:
  Returns:
    No return
  '''
  ! kaggle datasets download -d deividt/snake-classifier-pretrainedmodels #Download dataset
  ! unzip \*.zip && rm *.zip #Unzip downloaded file and delete zip

def kaggle_createDataset(public=False):
  '''
  Description:
    Creates dataset in Kaggle
  Parameters:
    public: Dataset Access:
  Returns:
    No return
  ''' 
  if public: 
    !kaggle datasets create -p  /content/model -u
  else: 
    !kaggle datasets create -p  /content/model

def kaggle_updateDataset(commit_message='Updating files'):
  '''
  Description:
    Update dataset in Kaggle
  Parameters:
    public: Dataset Access:
  Returns:
    No return
  ''' 
  !kaggle datasets version -p /content/model/ -m commit_message

def kaggle_setConfig(dataset_title="testing"):
  '''
  Description:
    Create dataset-metadata.json
  Parameters:
    dataset_title: Dataset name:
  Returns:
    No return
  ''' 
  fileName="data/dataset-metadata.json"
  !kaggle datasets init -p  /content/model/
  addTitle2JSON(fileName,dataset_title)


def kaggle_saveModel():
  '''
  Description:
    Saved pretrained model in kaggle
  Parameters:
    No parameters:
  Returns:
    No return
  ''' 
  res = input("Do you want upload model  in  your kaggle Account? [y/n] ")
  
  if  res.lower()!="y" :
    return print("Upload canceled")

  res = readOption( msg = "Create new dataset for saving models[1] or update old one[2]? [1/2]")
  
  dataset_title = input("Enter dataset name: ")
  assert len(res)>0 , "Invalid length "

  kaggle_setConfig(dataset_title)

  if res=="1": #Create new dataset
    res = readOption( msg = "Create dataset Private[1] or Public [2]? [1/2]" )
    public_dataset = True if int(res)==2 else False
    kaggle_createDataset(public_dataset)
  elif res=="2": #Update dataset
    commit_msg = input ("Enter commit message (Ex. 'version 1.4') : ")
    kaggle_updateDataset(commit_msg)



In [3]:
#! kaggle datasets download -d deividt/snake_breeds 
!kaggle datasets download -d deividt/snake-breeds

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


### Model Tools

In [4]:
def loadModel(name):
  '''
  Description:
    Load local model
  Parameters:
    No parameters:
  Returns:
    No return
  ''' 
  try:
    return load_model(name)
  except:
    raise Exception("Model not found")  

  

def fetchModel():
  '''
  Description:
  Download and set model
  Parameters:
  No parameters:
  Returns:
  model: tf.keras.Model
  ''' 
  print(bcolors.WARNING + " Downloading models from Kaggle..." + bcolors.ENDC)  

  fetchModelsFromKaggle()
  print(bcolors.OKGREEN + " Models downloaded in your local folder" + bcolors.ENDC)  
  print("---------------------------------")
  model_name = input ("\nEnter model file_name: ( Ex: model_a.h5 ) ")
  print(bcolors.WARNING + " Searching model..." + bcolors.ENDC)  

  m = loadModel(model_name)
  print(bcolors.OKGREEN + f"Model {model_name[:-3]} loaded successfully " + bcolors.ENDC)  
  return m

def getModel():
  '''
  Description:
  Create new model or download pre-trained model
  Parameters:
  No parameters:
  Returns:
  No return
  ''' 
  res = readOption(" Create new model [1] or load pre-trained model from Kaggle [2]  [1/2] ")

  if res =='1':
    model_name = input(" Set  model name: (Ex. model_A) ")
    model = createModel( name= model_name ,num_breeds = 10 )
    print(bcolors.OKGREEN + f"Model {model_name[:-3]} created successfully " + bcolors.ENDC)  
  else:
    if checkIfFileExist('kaggle.json')==False:
      loadCredentials()
    model = fetchModel();
  return model

def model2TFLite(model):

  '''
  Description:
    Convert model .h5 to .tflite
  Parameters:
    No parameters:
  Returns:
    No return 
  ''' 
  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tfmodel = converter.convert()
  open(model.name+'.tflite','wb').write(tfmodel)
  print("Model successfully convert to .tflite")


### Extra Tools

In [5]:
def readOption(msg):
  '''
  Description:
    Read user input
  Parameters:
    No parameters:
  Returns:
    No return 
  ''' 
  res = input(msg)
  assert res=='1' or res=='2',"Invalid option"
  return res

def decode_breedID(idx):
  df_decode = pd.read_csv('decode_breed.csv')
  breed_encode = df_decode['breed_encode']
  return df_decode.loc[breed_encode == idx]['breed'].tolist()[0]

def custom_preprocess_input(x):
  '''
  Description:
    Pre-process input from VGG16
  Parameters:
    x: numpy array
  Returns:
    x: numpy array 
  ''' 

  if not issubclass(x.dtype.type, np.floating):
    x = x.astype('float32', copy=False)
  print(x.dtype.type)

      # 'RGB'->'BGR'
 # x = x[..., ::-1]
  mean = [103.939, 116.779, 123.68]
  # Zero-center by mean pixel
  x[..., 0] -= mean[0]
  x[..., 1] -= mean[1]
  x[..., 2] -= mean[2]

  return x

### Training Tools

In [6]:
def train(model,x_train,y_train,x_test,y_test , batch_size, epochs, model_name=""):
    model.fit(x_train, y_train, epochs=epochs,
              batch_size=batch_size,
              validation_data=(x_test, y_test))
    metrics = model.evaluate(x_test, y_test)
    return {k:v for k,v in zip (model.metrics_names, metrics)}



### File Tools

In [7]:
def addTitle2JSON(fileName,dataset_title):
    data = readFile(fileName)
    data = addTitle2Data(data,dataset_title)
    writeFile(fileName,data)
    
def writeFile(fileName,data):
  with open(fileName,'w') as json_file:
      # Guardar la informacion
      json.dump(data,json_file)
      print("File "+fileName+" updated")

def readFile(fileName):
  with open(fileName, "r") as json_file: 
     return json.load(json_file)

def addTitle2Data(data,title):
  data['id'] = data['id'].split('/')[0]+'/'+title
  data['title'] = title
  return data

def checkIfFileExist(file_name):
    return os.path.exists(os.path.abspath(file_name))

## Convolutional network arquitecture

- Using VGG16 Arquitecture

In [84]:
#Custom model
def createModel(num_breeds,name="model", ) :
  from tensorflow.keras.layers import MaxPool2D 
  vgg= VGG16( 
      input_shape= (224,224,3), 
      weights='imagenet',
      include_top=False
      )
 
  #vgg.summary()
  custom_model=vgg


  for layer in custom_model.layers[:-6]:
    layer.trainable = False

  last_layer = MaxPool2D(pool_size=(4,4),strides=(2,2))(custom_model.layers[-6].output)
  last_layer = Flatten()(last_layer)
  last_layer = Dense(64, activation='relu', name='full_connected_1')(last_layer)
  last_layer = Dense(64, activation='relu', name='full_connected_2')(last_layer)
  out = Dense(num_breeds, activation='softmax', name='output')(last_layer)
  custom_model = Model(custom_model.input, out,name=name)

  
  opt = Adam(lr=0.001) #Stochastic gradient descent
  custom_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


  return custom_model

- Create a new model or load pre-trained model from Kaggle

In [85]:
m=createModel(10)
m.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [90]:
m.get_layer('block4_conv2').trainable

False

In [91]:
model = getModel()

 Create new model [1] or load pre-trained model from Kaggle [2]  [1/2] 1
 Set  model name: (Ex. model_A) model_B
Model mode created successfully 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


- Model summary

In [82]:
model.summary()

Model: "model_B"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [37]:
model=model.layers.remove(model.get_layer('block5_conv3'))
model.summary()

AttributeError: ignored

In [38]:
model.summary()

AttributeError: ignored

## Training and validation


- Fetch dataset from kaggle </br>

 - In order to use the Kaggle’s public API, you must first authenticate using an API token. For that follow the next steps: <br/> <br/>
     1. Go to https://www.kaggle.com/
     2. Click on your user profile picture
     3. Then on "Account" from the dropdown menu. This will take you yo your account settings
     4. Scroll down to the section of the page labelled API
     5. To create a new token, click on the “Create New API Token” button. This will download a fresh authentication token onto your machine named "kaggle.json"
     6.  <font color='red'>**YOU MUST UPLOAD kaggle.json FILE in the ROOT FOLDER**</font> 

In [11]:
fetchDatasetFromKaggle('snake-breeds',True)

Upload kaggle.json file with credentials

IMPORTANT: Filename must be: kaggle.json



Saving kaggle.json to kaggle.json
Data from kaggle successfully fetch

Unzipping data...
Archive:  snake-breeds.zip
  inflating: group_1.h5              
  inflating: group_10.h5             
  inflating: group_11.h5             
  inflating: group_12.h5             
  inflating: group_13.h5             
  inflating: group_14.h5             
  inflating: group_15.h5             
  inflating: group_16.h5             
  inflating: group_17.h5             
  inflating: group_18.h5             
  inflating: group_19.h5             
  inflating: group_2.h5              
  inflating: group_20.h5             
  inflating: group_21.h5             
  inflating: group_22.h5             
  inflating: group_23.h5             
  inflating: group_24.h5             
  inflating: group_25.h5             
  inflating: group_26.h5             
  inflating: group_27.h5             
  inflating: group_28.h5             
  inflating: group_29.h5             
  inflating: group_3.h5              
  inflatin

- Training and validate just certain amout of groups because of RAM limits

In this case is training just with eight groups

In [ ]:

for i in range(1,8):

    destination_filepath = 'group_'+str(i)+'.h5'
    with h5py.File(destination_filepath, "r") as f:
        
        print("Group:",i)
        
        x = f["input_data"][:]
        y = f["input_labels"][:]
        x=x.reshape(x.shape[0],224,224,3)
        x=preprocess_input(x*255)
        
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20)
    
    #Displaying information just for the first group
    if i ==1 :
      print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
      print("\nDistribution of train classes")
      print(pd.Series(y_train).value_counts())
      print("\nDistribution of test classes")
      print(pd.Series(y_test).value_counts())

    train(model,x_train,y_train,x_test,y_test , batch_size=32, epochs=10, model_name="modelA")

Group: 1
(1600, 224, 224, 3) (1600,) (400, 224, 224, 3) (400,)

Distribution of train classes
9    166
8    166
3    165
7    161
1    159
5    158
2    157
6    156
4    156
0    156
dtype: int64

Distribution of test classes
6    44
4    44
0    44
2    43
5    42
1    41
7    39
3    35
9    34
8    34
dtype: int64
Epoch 1/10
50/50 [==============================] - 1083s 22s/step - loss: 213.1004 - accuracy: 0.0950 - val_loss: 2.3049 - val_accuracy: 0.0875
Epoch 2/10
50/50 [==============================] - 1062s 21s/step - loss: 2.3012 - accuracy: 0.1050 - val_loss: 2.3048 - val_accuracy: 0.0850
Epoch 3/10
50/50 [==============================] - 1065s 21s/step - loss: 2.2990 - accuracy: 0.1056 - val_loss: 2.3059 - val_accuracy: 0.0850
Epoch 4/10
50/50 [==============================] - 1059s 21s/step - loss: 2.2967 - accuracy: 0.1063 - val_loss: 2.3075 - val_accuracy: 0.0850
Epoch 5/10
50/50 [==============================] - 1056s 21s/step - loss: 2.2939 - accuracy: 0.1037 - val

## Testing

-Testing with images from google

In [ ]:

#Set random image
img = cv2.imread('agkistrodon-contortrix.png')

img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC) #Resize
plt.imshow(img) #Display Image
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img= img.reshape(1,224,224,3)
img = custom_preprocess_input(img)
yhat = model.predict(img)

index = yhat.argmax()
print(decode_breedID( index ))
np.max(yhat)

## Saving model

- Local save

In [ ]:
#Saving model
!mkdir model
model.save('model/'model.name+".h5")

- Saving model in Kaggle

In [ ]:
kaggle_saveModel()

#Convert model to Tensorflow Lite

In [ ]:
model2TFLite(model)